In [2]:
from manim import *
#from classes.christmas_fourier_classes import *
from PIL import Image
import jupyter_capture_output

video_scene = f" -v WARNING --disable_caching sw_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s sw_Scene"

Manim Community v0.17.3

Jupyter Capture Output v0.0.11


In [76]:
GLOBAL_AMPLITUDE = 1
TUBE_LENGTH = 8
MAIN_COLOR = WHITE


# incoming wave
def get_wave_in(t, wave_params):
    lambda_wave = wave_params[0]
    omega_wave = wave_params[1]
    k_wave = 2*PI / lambda_wave
    v_ph = lambda_wave * omega_wave / 2 / PI
    def wave_in(x):
        # if (x < v_ph*t):
        #     return -np.sin(k_wave*x - omega_wave*t)
        # else:
        #     return 0
        return -np.sin(k_wave*x - omega_wave*t)
    return wave_in


# reflected wave
def get_wave_out(t, wave_params):
    lambda_wave = wave_params[0]
    omega_wave = wave_params[1]
    k_wave = 2*PI / lambda_wave
    v_ph = lambda_wave * omega_wave / 2 / PI
    def wave_out(x):
        # if (x < TUBE_LENGTH-omega_wave*t):
        #     return 0
        # else:
        #     return -np.sin(-k_wave*x - omega_wave*t - PI)
        return -np.sin(-k_wave*x - omega_wave*t)
    return wave_out


# superpositioned wave
def get_super_wave(t, wave_params):
    lambda_wave = wave_params[0]
    omega_wave = wave_params[1]
    k_wave = 2*PI / lambda_wave
    v_ph = lambda_wave * omega_wave / 2 / PI
    T = TUBE_LENGTH / v_ph
    if (t < T):
        def super_wave(x):
            if (x < v_ph*t):
                return get_wave_in(t, wave_params)(x)
            else:
                return 0
        return super_wave
    elif (t < 2*T):
        def super_wave(x):
            if (x < TUBE_LENGTH - v_ph*(t-T)):
                return get_wave_in(t, wave_params)(x)
            else:
                return get_wave_in(t, wave_params)(x) + get_wave_out(t, wave_params)(x)
        return super_wave
    else:
        def super_wave(x):
            return get_wave_in(t, wave_params)(x) + get_wave_out(t, wave_params)(x)
        return super_wave


# class creating a box to display incoming / outgoing transversal wave as well as their superposition
class StandingWaveTrans(Mobject):
    def __init__(self, position, height, width, **kwargs):
        super().__init__(**kwargs)
        self.vertical_shift = np.array([0, -height/6, 0])
        rectangle = Rectangle(height = height, width = width, stroke_width = 2, color = MAIN_COLOR).move_to(position)
        separator = Line(start = position - np.array([width/2, 0, 0]), end = position + np.array([width/2, 0, 0]), stroke_width = 1, color = MAIN_COLOR).shift(self.vertical_shift)
        self.add(rectangle, separator)
        text_super = Tex(r"Superposition", font_size = 24, color = MAIN_COLOR).next_to(separator, 0.5*DOWN + LEFT).shift(height/2 * UP).shift(width * RIGHT).shift(-self.vertical_shift)
        text_individual = Tex(r"Einzelwellen", font_size = 24, color = MAIN_COLOR).next_to(separator, 0.5*DOWN + LEFT).shift(width * RIGHT)
        self.add(text_super, text_individual)
        # number plane for the superposition
        self.super_npla = NumberPlane(
            x_range = [0, TUBE_LENGTH, 1], y_range = [-3, 3, 1], x_length = width, y_length = 9/10*height*4/6,
            x_axis_config = {"stroke_opacity": 0.215, "stroke_color": MAIN_COLOR}, y_axis_config = {"stroke_opacity": 0.125, "stroke_color": MAIN_COLOR}, background_line_style = {"stroke_opacity": 0.125}
        ).move_to(position + np.array([0, height/4, 0])).shift(self.vertical_shift/2)
        self.add(self.super_npla)
        # number plane for the individual waves
        self.individual_npla = NumberPlane(
            x_range = [0, TUBE_LENGTH, 1], y_range = [-1.75, 1.75, 1], x_length = width, y_length = 9/10*height*2/6,
            x_axis_config = {"stroke_opacity": 0.215, "stroke_color": MAIN_COLOR}, y_axis_config = {"stroke_opacity": 0.125, "stroke_color": MAIN_COLOR}, background_line_style = {"stroke_opacity": 0.125}
        ).move_to(position - np.array([0, height/4, 0])).shift(self.vertical_shift/2)
        self.add(self.individual_npla)


    def get_incoming(self, t, wave_params):
        lambda_wave = wave_params[0]
        omega_wave = wave_params[1]
        v_ph = lambda_wave * omega_wave / 2 / PI
        return self.individual_npla.plot(get_wave_in(t, wave_params), x_range = [0, min(v_ph*t, TUBE_LENGTH)], stroke_width = 2, stroke_color = RED)

    def get_reflected(self, t, wave_params):
        lambda_wave = wave_params[0]
        omega_wave = wave_params[1]
        v_ph = lambda_wave * omega_wave / 2 / PI
        T = TUBE_LENGTH / v_ph
        if (t > T):
            return self.individual_npla.plot(get_wave_out(t, wave_params), x_range = [max(0, T*v_ph + TUBE_LENGTH - v_ph*t), TUBE_LENGTH], stroke_width = 2, stroke_color = BLUE)
        else:
            return Dot(fill_opacity = 0)

    def get_superposition(self, t, wave_params):
        lambda_wave = wave_params[0]
        omega_wave = wave_params[1]
        v_ph = lambda_wave * omega_wave / 2 / PI
        return self.super_npla.plot(get_super_wave(t, wave_params), x_range = [0, TUBE_LENGTH], stroke_width = 2, stroke_color = GREY)

In [77]:
%%capture_video --path "animations/standing_wave/standing_wave.mp4"
%%manim -qm --fps 60 $video_scene


class sw_Scene(Scene):
    def construct(self):
        #self.camera.background_color = WHITE
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        # Wellenparameter
        n = 3
        LAMBDA_WAVE = (4 * TUBE_LENGTH) / (2*n - 1)
        OMEGA_WAVE = 2
        wave_params = (LAMBDA_WAVE, OMEGA_WAVE)

        wave_trans_box = StandingWaveTrans(position = np.array([0, 0, 0]), height = 4, width = 6)
        self.add(wave_trans_box)

        incoming_wave = wave_trans_box.get_incoming(0, wave_params)
        #incoming_wave.params = wave_params
        incoming_wave.get_method = wave_trans_box.get_incoming
        self.add(incoming_wave)

        reflected_wave = wave_trans_box.get_reflected(0, wave_params)
        #reflected_wave.params = wave_params
        reflected_wave.get_method = wave_trans_box.get_reflected
        self.add(reflected_wave)

        superpositioned_wave = wave_trans_box.get_superposition(0, wave_params)
        #superposition_wave.params = wave_params
        superpositioned_wave.get_method = wave_trans_box.get_superposition
        self.add(superpositioned_wave)


        # updates the waves using the ValueTracker time 't'
        def wave_updater(wave):
            t = t_tracker.get_value()
            #wave_params = wave.params
            wave.become(wave.get_method(t, wave_params))


        self.wait(1)
        t_tracker = ValueTracker(0)
        incoming_wave.add_updater(wave_updater)
        reflected_wave.add_updater(wave_updater)
        superpositioned_wave.add_updater(wave_updater)
        self.play(t_tracker.animate.set_value(10), rate_func = linear, run_time = 10)
        self.wait(5)

Output saved by overwring previous file at animations/standing_wave/standing_wave.mp4.
